This notebook contains the code for the EDA for CO2 Emissions by Country. 
Explanations for the code will be provided with markdown cells if applicable. More in depth reasonings and theory can be found in the thesis.

**What we are measuring written below:**
Mean value  
Median value 
normal distribution 
Relations between columns
Top 10 highest CO2 Emissions by Country
Top 10 lowest CO2 Emissions by Country

**Cleaning methodologies written below:**
extreme values with max and min values (0,1%  1%  2,5%  5%)
raw (no cleaning)
Remove row with empty value (null)
Replace empty values with the mean value **check this later**
Normalize

**Metrics for each column**
This was not included in the documentation so these where researched and added here.  
'Year', 'Total', 'Coal', 'Oil', 'Gas','Cement', 'Flaring', 'Other', = Megatonnes 
per capita = tons
per caipta example: Austria 2014 61,16 megatonnes total devided by 8 546 000 (official population estimate with a simple google search) gets 7,507605 with the dataset pointing at 7.507548. 

In [16]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

try:
    dataCo2 = pd.read_csv('CO2.csv')
    dataCo2= pd.DataFrame(dataCo2)
except:
    FileNotFoundError 
    print("data can´t be found")

# Rename coulmn names to be more readable
dataCo2=dataCo2.rename({"ISO 3166-1 alpha-3": "ISO Code", "Total":"Total Co2 Emission (Mt)", "Per Capita":"Per Capita (t)"},axis=1)
print(dataCo2.columns,"\n")


Index(['Country', 'ISO Code', 'Year', 'Total Co2 Emission (Mt)', 'Coal', 'Oil',
       'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita (t)'],
      dtype='object') 



In [17]:
print("number of nan values per column\n",dataCo2.isna().sum()) # check for nan values.

number of nan values per column
 Country                        0
ISO Code                    1632
Year                           0
Total Co2 Emission (Mt)      200
Coal                       41360
Oil                        41387
Gas                        41486
Cement                     42290
Flaring                    41554
Other                      61484
Per Capita (t)             44130
dtype: int64
